In [125]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.affinity import translate

In [126]:
# 1. Load centroids
centroids_gdf = gpd.read_file("../../data/czech/czech_centriods.gpkg")

In [127]:
# 2. Select base point for legend (Středočeský kraj)
base_point = centroids_gdf[centroids_gdf["nazev"] == "Středočeský kraj"].geometry.values[0]

In [128]:
# 3. Create vertical legend points (stacked 50 km apart)
legend_vertical = gpd.GeoDataFrame([
    {"geometry": base_point, "prichozi": 1001, "odchozi": 1000, "celkem": 2001},
    {"geometry": translate(base_point, xoff=0, yoff=-25000), "prichozi": 1000, "odchozi": 1001, "celkem": 2001}
], crs=centroids_gdf.crs)

print(legend_vertical)

                          geometry  prichozi  odchozi  celkem
0  POINT (4623222.961 2973912.777)      1001     1000    2001
1  POINT (4623222.961 2948912.777)      1000     1001    2001


In [129]:
# 4. Create horizontal legend points (spaced 60 km apart)
point_left   = translate(base_point, xoff=0, yoff=-50000)
point_middle = translate(base_point, xoff=25000, yoff=-50000)
point_right  = translate(base_point, xoff=50000, yoff=-50000)

In [130]:
# 2. Get total values from selected regions
total_kk  = centroids_gdf[centroids_gdf["nazev"] == "Karlovarský kraj"]["celkem"].values[0]
total_stc = centroids_gdf[centroids_gdf["nazev"] == "Středočeský kraj"]["celkem"].values[0]
total_prg = centroids_gdf[centroids_gdf["nazev"] == "Hlavní město Praha"]["celkem"].values[0]

In [131]:
legend_horizontal = gpd.GeoDataFrame([
    {"geometry": point_left,   "celkem": total_kk},
    {"geometry": point_middle, "celkem": total_stc},
    {"geometry": point_right,  "celkem": total_prg}
], crs=centroids_gdf.crs)
print(legend_horizontal)

                          geometry  celkem
0  POINT (4623222.961 2923912.777)    5196
1  POINT (4648222.961 2923912.777)   35056
2  POINT (4673222.961 2923912.777)   75229


In [132]:
# 2. Vytvoříme základní linii a posuneme ji dolů
#○point_right = Point(point_right.x, point_right.y + 1000.001)
full_line = LineString([point_left, point_right])
full_line_shifted = translate(full_line, xoff=0, yoff=-25000)

In [133]:
# 3. Funkce pro oříznutí linie podle procenta
def trim_line(line, pct):
    target_length = line.length * pct
    trimmed_coords = []
    current_length = 0

    coords = list(line.coords)
    for i in range(len(coords) - 1):
        start = coords[i]
        end = coords[i + 1]
        segment = LineString([start, end])
        seg_length = segment.length

        if current_length + seg_length < target_length:
            trimmed_coords.append(start)
            current_length += seg_length
        else:
            remaining = target_length - current_length
            point = segment.interpolate(remaining)
            trimmed_coords.append(start)
            trimmed_coords.append((point.x, point.y))
            break

    if len(trimmed_coords) < 2:
        return LineString([coords[0], coords[0]])
    return LineString(trimmed_coords)

In [134]:
# 4. Vytvoříme oříznuté linie a také je posuneme
line_50l = translate(trim_line(full_line, 0.50), xoff=0, yoff=-25000)
line_50r = translate(trim_line(full_line, 0.50), xoff=25000, yoff=-25000)
line_25l = translate(trim_line(full_line, 0.25), xoff=0, yoff=-50000)
line_25r = translate(trim_line(full_line, 0.75), xoff=12500, yoff=-50000)
line_75l = translate(trim_line(full_line, 0.75), xoff=0, yoff=-75000)
line_75r = translate(trim_line(full_line, 0.25), xoff=37500, yoff=-75000)

In [135]:
# 5. Sestavíme GeoDataFrame
legend_lines = gpd.GeoDataFrame([
    {"geometry": line_50l, "nazev": "50l", "typ": "l"},
    {"geometry": line_50r, "nazev": "50r", "typ": "r"},
    {"geometry": line_25l, "nazev": "25l", "typ": "l"},
    {"geometry": line_25r, "nazev": "25r", "typ": "r"},
    {"geometry": line_75l, "nazev": "75l", "typ": "l"},
    {"geometry": line_75r, "nazev": "75r", "typ": "r"},
], crs=legend_horizontal.crs)

In [136]:
# 5. Save both to a single GeoPackage
output_path = "legend_combined.gpkg"
legend_vertical.to_file(output_path, layer="legend_vertical", driver="GPKG")
legend_horizontal.to_file(output_path, layer="legend_horizontal", driver="GPKG")
legend_lines.to_file(output_path, layer="legend_lines", driver="GPKG")